<a href="https://colab.research.google.com/github/viksimenaite/NaturalLanguageModel/blob/main/2_GMM_Lab_darbas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Roberta model

In [1]:
!pip install simpletransformers

     |████████████████████████████████| 225kB 22.6MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 1.8MB 47.3MB/s 
     |████████████████████████████████| 8.2MB 47.6MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 2.3MB 53.2MB/s 
     |████████████████████████████████| 122kB 57.0MB/s 
     |████████████████████████████████| 1.2MB 50.1MB/s 
     |████████████████████████████████| 3.3MB 46.3MB/s 
     |████████████████████████████████| 235kB 57.5MB/s 
     |████████████████████████████████| 102kB 13.1MB/s 
     |████████████████████████████████| 174kB 57.0MB/s 
     |████████████████████████████████| 133kB 55.6MB/s 
     |████████████████████████████████| 112kB 60.2MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 4.2MB 38.1MB/s 
     |████████████████████████████████| 901kB 49.7MB/s 
     |████████████████████████████████| 245kB 58.0MB

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
from simpletransformers.question_answering import QuestionAnsweringModel
import numpy as np
import json
import math
import torch

# Taking training and evaluation data

In [12]:
with open('/content/drive/MyDrive/DataSet/train-v2.0.json', 'r') as file:
    training_data_set = json.load(file)

training_data_set = [item for topic in training_data_set['data'] for item in topic['paragraphs'] ]

with open('/content/drive/MyDrive/DataSet/dev-v2.0.json', 'r') as file:
    evaluation_data_set = json.load(file)

evaluation_data_set = [item for topic in evaluation_data_set['data'] for item in topic['paragraphs'] ]
#print(len(evaluation_data_set))

# Divide data sets into smaller chunks

In [13]:
def divided_data_chunk(data_set, chunk_number):
    n = math.ceil(len(data_set)/chunk_number)

    for x in range(0, len(data_set), n):
        chunk = data_set[x: n+x]

        if len(chunk) < n:
            chunk = chunk + [None for y in range(n-len(chunk))]
        yield chunk

training_chunks = list(divided_data_chunk(training_data_set, chunk_number=1))

eval_chunks = list(divided_data_chunk(evaluation_data_set, 3))

# Model training

In [14]:
is_cuda_available = torch.cuda.is_available()

training_args = {
    'num_train_epochs': 2,
    'train_batch_size': 25,
}

model = QuestionAnsweringModel('roberta', 'roberta-base', args=training_args, use_cuda=is_cuda_available)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use 

In [ ]:
model.train_model(training_chunks[0], output_dir='/content/drive/MyDrive/training', show_running_loss=True)

add example index and unique id: 100%|██████████| 130319/130319 [00:00<00:00, 876503.95it/s]


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/5258 [00:01<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running Epoch 1 of 2:   0%|          | 0/5258 [00:00<?, ?it/s]

# Evaluating trained model

In [ ]:
trained_model = QuestionAnsweringModel('roberta', '/content/drive/MyDrive/training/outputs', args=training_args, use_cuda=is_cuda_available)

In [ ]:
trained_model.eval_model(eval_chunks[0], output_dir = '/content/drive/MyDrive/evaluating')

# Predicting

One question predicting

In [ ]:
context_text = "Mary got the milk there. John moved to the bedroom. Sandra went back to the kitchen. Mary travelled to the hallway."
question = "Where is the milk ?"

predictions = trained_model.predict(
    [
        {
            "context": context_text,
            "qas": [
                {
                    "question": question,
                    "id": "0",
                }
            ],
        }
    ]
)

print(predictions)

Multiple questions

In [ ]:
path = '/content/drive/MyDrive/'
questions = []
q_id = 0
with open(path + 'answers.txt', 'w') as writefile:
  with open(path + 'task_2_test_set_questions.txt', "r") as filestream:
    for line in filestream:
      currentline = line.split(",")

      predictions = trained_model.predict([
        {
            "context": currentline[0],
                "qas": [
                    {
                        "question": currentline[1].replace('\n', ''),
                        "id": q_id,
                    }
                ],
        }
      ])
      answer = predictions[0]['answer'][1]
      writefile.write(str(answer) + '\n')
      q_id += 1
